In [3]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2

In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
!unzip './IMAGES' -d '/content/'

unzip:  cannot find either './IMAGES' or './IMAGES'.zip.


In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    './IMAGES',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    './IMAGES',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 11152 images belonging to 2 classes.
Found 2788 images belonging to 2 classes.


In [5]:
def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1 - y_true) * (1 - y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return K.mean(f1)

In [6]:
# Check for GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Load pre-trained model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create model
model = Model(inputs=base_model.input, outputs=predictions)

Num GPUs Available:  1


In [7]:

# Freeze base model
base_model.trainable = False

# Compile model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy', f1_score])

# Callbacks for learning rate reduction and early stopping
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the added layers without the base model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10
)

Epoch 1/10
349/349 [==============================] - 185s 510ms/step - loss: 0.6993 - accuracy: 0.5144 - f1_score: 0.2187 - val_loss: 0.6964 - val_accuracy: 0.4706 - val_f1_score: 0.6290
Epoch 2/10
349/349 [==============================] - 178s 509ms/step - loss: 0.6940 - accuracy: 0.5156 - f1_score: 0.1829 - val_loss: 0.6950 - val_accuracy: 0.5294 - val_f1_score: 0.0000e+00
Epoch 3/10
349/349 [==============================] - 176s 503ms/step - loss: 0.6925 - accuracy: 0.5237 - f1_score: 0.1137 - val_loss: 0.6918 - val_accuracy: 0.5294 - val_f1_score: 0.0000e+00
Epoch 4/10
349/349 [==============================] - 169s 484ms/step - loss: 0.6917 - accuracy: 0.5294 - f1_score: 0.0000e+00 - val_loss: 0.6915 - val_accuracy: 0.5294 - val_f1_score: 0.0000e+00
Epoch 5/10
349/349 [==============================] - 167s 479ms/step - loss: 0.6915 - accuracy: 0.5297 - f1_score: 0.0013 - val_loss: 0.6914 - val_accuracy: 0.5294 - val_f1_score: 0.0000e+00
Epoch 6/10
349/349 [====================

In [8]:
# Fine_Tuning
    
# Unfreeze some layers and fine-tune

base_model.trainable = True

#for layer in base_model.layers[:]:
    #layer.trainable = True

# Recompile with lower learning rate
model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy', f1_score])

# Continue training
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15
    #callbacks=[reduce_lr, early_stopping]
)

Epoch 1/15


ResourceExhaustedError: Graph execution error:

Detected at node 'model/block3a_bn/FusedBatchNormV3' defined at (most recent call last):
    File "C:\Users\CBE\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\CBE\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\CBE\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\CBE\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\CBE\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\CBE\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platform\as

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

# If you have recorded F1 scores, plot them as well
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['f1_score'])
plt.plot(history.history['val_f1_score'])
plt.title('Model F1 Score')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

# Get the true labels and predictions
y_true = test_generator.classes
y_pred = model.predict(test_generator)
y_pred = np.round(y_pred).flatten()

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot(cmap=plt.cm.Blues)
plt.show()


In [ ]:
from keras.models import load_model
# Save the model
model.save('my_1st_model.h5')
# Load the model
loaded_model = load_model('my_1st_model.h5')

In [ ]:
# Evaluate model
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'path/to/test_data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_accuracy, test_f1_score = model.evaluate(test_generator)
print(f'test_loss: {test_loss:.2f}')
print(f'Test accuracy: {test_accuracy:.2f}')
print(f'test_f1_score: {test_f1_score:.2f}')

NameError: name 'ImageDataGenerator' is not defined